# 882 Replication Paper: Flabbi 2010

- Data from CPS, 1995
- Estimation strategy from Flabbi 2010 

## Import Packages

In [1]:
# Data Manipulation 
import numpy as np
import pandas as pd

# General
import pdb

# Estimation
from scipy.optimize import minimize
import scipy.stats as stats
# import numdifftools as ndt

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn.apionly as sns
from pylab import *

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


## Import Data 

- CPS data on gender, wages, duration of unemployment
- M: males
- F: females
- U: unemployed
- E: employed 
- {M,F} X {E,U} = {males,females} X {employed, unemployed}

In [2]:
data=pd.read_csv('../data/est_c.csv')
data.columns = ['dur', 'wage', 'empl', 'women']

M = data[data['women']==0] #1186 men 
F = data[data['women']==1] #993 women 
U = data[data['empl']==0] #45 unemployed
E = data[data['empl']==1] #2134 employed

ME = M[M['empl']==1] #1168 employed men
MU = M[M['empl']==0] #18 unemployed men
FE = F[F['empl']==1] #966 employed women
FU = F[F['empl']==0] #27 unemployed women

### Summary Statistics, without trimming

In [ ]:
agg_dict = {
    'wage': ['mean', 'std', 'count'],
    'dur': ['mean', 'std', 'count'],
}

In [ ]:
print(data.groupby(['women', 'empl']).agg(agg_dict).to_latex()) # by gender

In [ ]:
print(data.groupby(['empl']).agg(agg_dict).to_latex()) # all

### Summary Statistics, with trimming

In [ ]:
print(np.percentile(ME['wage'], 5))

In [ ]:
print(np.percentile(FE['wage'], 5))

In [3]:
M_WAGE_COND = (data['wage'] > 7.27) #hard coded percentile so it does not continually update
M_COND = (data['women'] == 0)

F_WAGE_COND = (data['wage'] > 5.75625) #hard coded percentile so it does not continually update
F_COND = (data['women'] == 1)

DUR_COND = (data['dur'] > 0)

trim = data[ (M_COND & M_WAGE_COND) | (F_COND & F_WAGE_COND) | DUR_COND ]


M = trim[trim['women']==0] #1127 men 
F = trim[trim['women']==1] #944 women 
U = trim[trim['empl']==0] #45 unemployed
E = trim[trim['empl']==1] #2026 employed

ME = M[M['empl']==1] #1109 employed men
MU = M[M['empl']==0] #18 unemployed men
FE = F[F['empl']==1] #917 employed women
FU = F[F['empl']==0] #27 unemployed women

In [ ]:
print(trim.groupby(['women', 'empl']).agg(agg_dict).to_latex()) # by gender

In [ ]:
print(trim.groupby(['empl']).agg(agg_dict).to_latex()) # all

## Figures 

- Distribution of wages, men and women

In [ ]:
# Seaborn
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

sns.distplot(ME['wage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=50, ax=ax[0])
sns.distplot(FE['wage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=50, ax=ax[1])

ax[0].legend(['Men'])
ax[1].legend(['Women'])

ax[0].set_xlim([0,75])
ax[1].set_xlim([0,75])

ax[0].set_ylim([0,0.07])
ax[1].set_ylim([0,0.07])

plt.tight_layout()

In [ ]:
fig.savefig('./figures/fig1_2.png', bbox_inches='tight', transparent=True)

## Estimation 

### Initial Conditions

In [4]:
λ_0 = 0.22
λ_M = 0.18
λ_F = 0.28

h_0 = 0.234
h_M = 0.203
h_F = 0.260

η_0 = 0.005
η_M = 0.003
η_F = 0.0077

μ_0 = 3.433
μ_M = 3.456
μ_F = 3.454

σ_0 = 0.523
σ_M = 0.558
σ_F = 0.423

p_0 = 0.5
d_0 = 15 #leads to successful termination (13 leads to unsuccessful)

### Parameters without distributional assumptions

In [5]:
wstarM = min(ME['wage'])
wstarF = min(FE['wage'])

α = 0.5

### Accepted Wage Distribution as in 13

In [6]:
def dens_accepted(wage: np.array, α: float, μ: float, σ: float, wstar: float):
    """
    Calculates the density of accepted wages using the lognormal distribution (end of eq 13)
    """
    
    pdf_in = (wage - (1-α)*wstar)/α
    sf_in = wstar
    
    shape = 1
    
    num = stats.lognorm.pdf(pdf_in, shape, μ, σ) 
    denom = stats.lognorm.sf(sf_in, shape, μ, σ)
    
    return num / (α * denom)

In [7]:
def dens_accepted_prej(wage: np.array, α: float, μ: float, σ: float, wstar: float, d: float):
    """
    Calculates the density of accepted wages when prejudice is present using the lognormal distribution (end of eq 13)
    """
    
    shape = 1
    
    pdf_in = (wage + α*d - (1-α)*wstar)/α
    sf_in = wstar+d
    
    num = stats.lognorm.pdf(pdf_in, shape, μ, σ) 
    denom = stats.lognorm.sf(sf_in, shape, μ, σ)    
    
    return num / (α * denom)

In [8]:
def hazardM(λ: float, wstar: float, α: float, μ: float, σ: float):
    """
    Estimates hazard rate for men (eq 10 of the paper)
    """
    
    shape = 1

    mult = stats.lognorm.sf(wstar, shape, μ, σ)
    
    return λ * mult

In [9]:
def hazardF(λ: float, wstar: float, α: float, μ: float, σ: float, p: float, d: float):
    """
    Estimates hazard rate for women (eq 10 of the paper)
    """
    
    shape = 1
    
    sf_in1 = wstar
    sf_in2 = wstar+d
    
    mult = (1-p)*stats.lognorm.sf(sf_in1, shape, μ, σ) + p*stats.lognorm.sf(sf_in2, shape, μ, σ)
    
    return λ * mult

### Test Statistics

In [20]:
def teststats (hess_inv: np.ndarray, lnL: float, nparams: int):
    """
    Calculates the standard errors and p value from the LR tests
    """
    se = np.sqrt(np.diag(hess_inv))
    
    loglik_H0 = lnL_6
    
    LR = 2 * (lnL - loglik_H0)
    pval = stats.chi2.pdf(LR, nparams)
    
    return [se.tolist(), [lnL, pval]]

### Likelihood Functions

#### Estimation 6

In [11]:
def loglik_6( params: list ):
    """
    Calculates log likelihood with prejudice and productivity differences (Estimation 6)
    
    Parameters to estimate: λM, λF, ηM, ηF, μM, σM, μF, σF, d, p
    """
    
    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
#     μM = np.exp(params[4])
    μM = params[4]
    σM = np.exp(params[5])
#     μF = np.exp(params[6])
    μF = params[6]
    σF = np.exp(params[7])
    d  = np.exp(params[8])
    p  = np.exp(params[9])
#     p  = np.exp(params[9])/(1+np.exp(params[9]))
    
    # Men's equations 
    hM = hazardM(λM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted( ME['wage'], α, μM, σM, wstarM) ) )  

    
    # Women's equations
    hF = hazardF(λF, wstarF, α, μF, σF, p, d)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    h = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted( FE['wage'], α, μF, σF, wstarF)
    z = (p/α) * dens_accepted_prej( FE['wage'], α, μF, σF, wstarF, d)
      
    i = np.sum( np.log( y + z ) )
    
    
    return -(a + b + c + e + f + g + h + i)

In [12]:
# Check on log-likelihood

param6 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F, d_0, p_0]

b6_0 = np.log(param6[0])
b6_1 = np.log(param6[1])
b6_2 = np.log(param6[2])
b6_3 = np.log(param6[3])
# b6_4 = np.log(param6[4])
b6_4 = param6[4]
b6_5 = np.log(param6[5])
# b6_6 = np.log(param6[6])
b6_6 = param6[6]
b6_7 = np.log(param6[7])
b6_8 = np.log(param6[8])
b6_9 = np.log(1)

init6 = [b6_0, b6_1, b6_2, b6_3, b6_4, b6_5, b6_6, b6_7, b6_8, b6_9]

print(loglik_6(init6))

17645.414209251347


In [13]:
print(init6)

[-1.7147984280919266, -1.2729656758128873, -5.809142990314028, -4.866534950122499, 3.456, -0.583396316600826, 3.454, -0.8603830999358592, 2.70805020110221, 0.0]


In [14]:
est_6 = minimize(loglik_6, init6)

est_6

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/

      fun: 6115.957426865454
 hess_inv: array([[ 3.73780874e-02, -1.60028020e-02,  2.86912794e-02,
        -2.39748049e-02, -4.20010641e-03,  2.15635505e-04,
        -4.84713297e-03,  1.32040319e-04, -2.95718560e-04,
        -7.58494909e-04],
       [-1.60028020e-02,  2.97712970e-02, -2.29497417e-02,
         2.58592723e-02, -3.49378885e-04,  2.68635011e-06,
         1.37706455e-03, -8.40299529e-04, -8.46528369e-05,
        -2.58747565e-04],
       [ 2.86912794e-02, -2.29497417e-02,  6.45895884e-02,
        -3.42837679e-02, -5.52233022e-03,  4.06509636e-04,
        -1.14724856e-03,  1.26254767e-04,  4.36157918e-05,
        -8.04438583e-04],
       [-2.39748049e-02,  2.58592723e-02, -3.42837679e-02,
         5.60944028e-02, -4.62873392e-04, -5.69274342e-05,
        -3.46048654e-03, -9.00099498e-04, -5.06596361e-04,
        -2.33722182e-04],
       [-4.20010641e-03, -3.49378885e-04, -5.52233022e-03,
        -4.62873392e-04,  3.34289643e-02, -1.92096253e-03,
         3.51274832e-03, -4.99

In [15]:
# Coefficients

λM_6 = np.exp(est_6.x[0])
λF_6 = np.exp(est_6.x[1])
ηM_6 = np.exp(est_6.x[2])
ηF_6 = np.exp(est_6.x[3])
# μM_6 = np.exp(est_6.x[4])
μM_6 = est_6.x[4]
σM_6 = np.exp(est_6.x[5])
# σM_6 = est_6.x[5]
# μF_6 = np.exp(est_6.x[6])
μF_6 = est_6.x[6]
σF_6 = np.exp(est_6.x[7])
# σF_6 = est_6.x[7]
d_6  = np.exp(est_6.x[8])
p_6  = np.exp(est_6.x[9])
# p_6  = np.exp(est_6.x[9])/(1+np.exp(est_6.x[9]))

coeff_6 = [λM_6, λF_6, ηM_6, ηF_6, μM_6, σM_6, μF_6, σF_6, d_6, p_6]

print(coeff_6)

[0.20318982948244704, 0.2607853385960906, 0.0032968838821281813, 0.007672429654633786, 6.627589789784699, 23.469269248327624, 16.748203814377177, 12.354384836155761, 11.626858273307368, 0.48174094703035725]


In [16]:
lnL_6 = est_6.fun

print(lnL_6)

6115.957426865454


In [21]:
ts_6 = teststats(est_6.hess_inv, lnL_6, 10)

print(ts_6)

[[0.19333413396458246, 0.1725436090925145, 0.25414481775174763, 0.2368425697080135, 0.18283589444274412, 0.03050634469918594, 0.18270072522255384, 0.042221981524450745, 0.02318536953649895, 0.0612477340140531], [6115.957426865454, 0.0]]


#### Estimation 3

In [22]:
def loglik_3( params: list ):
    """
    Calculates log likelihood with prejudice and productivity differences 
    
    Estimation 3 (η and h not gender specific)
    
    Parameters to estimate: λ, η, μM, σM, μF, σF, d, p
    """
    
    λM = np.exp(params[0]) #same lambda
    λF = np.exp(params[0]) #same lambda
    η = np.exp(params[1]) #same eta
#     μM = np.exp(params[2])
    μM = params[2]
    σM = np.exp(params[3])
#     μF = np.exp(params[4])
    μF = params[4]
    σF = np.exp(params[5])
    d = np.exp(params[6])
    p = np.exp(params[7])
    
    # Men's equations
    hM = hazardM(λM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+η))
    b = MU['dur'].count() * np.log(η)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted( ME['wage'], α, μM, σM, wstarM) ) )
    
    
    # Women's equations
    hF = hazardF(λF, wstarF, α, μM, σM, p, d)
    
    f = F['dur'].count() * np.log(hF/(hF+η))
    g = FU['dur'].count() * np.log(η)
    i = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted( FE['wage'], α, μF, σF, wstarF)
    z = (p/α) * dens_accepted_prej( FE['wage'], α, μF, σF, wstarF, d)

    j = np.sum( np.log( y + z ) )
    
    
    return -(a + b + c + e + f + g + i + j)

In [23]:
# Check on log-likelihood

param3 = [λ_0, η_0, μ_M, σ_M, μ_F, σ_F, d_0, p_0]

b3_0 = np.log(param3[0])
b3_1 = np.log(param3[1])
# b3_2 = np.log(param3[2])
b3_2 = param3[2]
b3_3 = np.log(param3[3])
# b3_4 = np.log(param3[4])
b3_4 = param3[4]
b3_5 = np.log(param3[5])
b3_6 = np.log(param3[6])
b3_7 = np.log(1)

init3 = [b3_0, b3_1, b3_2, b3_3, b3_4, b3_5, b3_6, b3_7]

print(loglik_3(init3))

16629.36100816613


In [24]:
est_3 = minimize(loglik_3, init3)

est_3

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launc

      fun: 6130.3380799291535
 hess_inv: array([[ 4.48544393e-02,  2.17557825e-02,  1.57936036e-02,
         1.29214233e-03,  5.92618122e-01,  1.98794170e-02,
         5.51051975e-02,  7.05413079e-02],
       [ 2.17557825e-02,  4.11046254e-02,  4.82092772e-03,
         7.19001662e-05,  1.27220285e-01,  4.50406025e-03,
         1.20610505e-02,  1.50775908e-02],
       [ 1.57936036e-02,  4.82092772e-03,  4.08202519e-02,
        -9.40704421e-04,  3.23128215e-01,  1.04626066e-02,
         3.02683683e-02,  3.85022876e-02],
       [ 1.29214233e-03,  7.19001662e-05, -9.40704421e-04,
         9.99541166e-04,  3.29329774e-02,  1.10122291e-03,
         3.07645445e-03,  3.89403019e-03],
       [ 5.92618122e-01,  1.27220285e-01,  3.23128215e-01,
         3.29329774e-02,  1.18741116e+01,  4.10753774e-01,
         1.10826783e+00,  1.40980285e+00],
       [ 1.98794170e-02,  4.50406025e-03,  1.04626066e-02,
         1.10122291e-03,  4.10753774e-01,  1.45794487e-02,
         3.83823621e-02,  4.90396816

In [25]:
# Coefficients

λ_3 = np.exp(est_3.x[0])
η_3 = np.exp(est_3.x[1])

# μM_3 = np.exp(est_3.x[2])
# σM_3 = np.exp(est_3.x[3])
# μF_3 = np.exp(est_3.x[4])
# σF_3 = np.exp(est_3.x[5])

μM_3 = est_3.x[2]
σM_3 = est_3.x[3]
μF_3 = est_3.x[4]
σF_3 = est_3.x[5]

d_3 = np.exp(est_3.x[6])
p_3 = np.exp(est_3.x[7])

coeff_3 = [λ_3, η_3, μM_3, σM_3, μF_3, σF_3, d_3, p_3]

print(coeff_3)

[0.270755782189324, 0.0053674232268518, 6.696061456553306, 3.1611807294646583, 15.203276380543914, 2.706358724009457, 10.474295653012167, 0.5274219414478007]


In [26]:
lnL_3 = est_3.fun

print(lnL_3)

6130.3380799291535


In [27]:
ts_3 = teststats(est_3.hess_inv, lnL_3, 8)

print(ts_3)

[[0.21178866655894932, 0.20274275665779656, 0.2020402235204598, 0.0316155209746657, 3.445883284298531, 0.12074538809047451, 0.32448758235783465, 0.4101052213208973], [6130.3380799291535, 0.0001408368519992885]]


#### Estimation 5

In [28]:
def loglik_5( params: list ):
    """
    Calculates log likelihood with prejudice, no productivity differences (estimation 5). 
    
    Parameters to estimate: λM, λF, μ, σ, p, d
    """

    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
#     μ = np.exp(params[4])
    μ = params[4]
    σ = np.exp(params[5])
    d = np.exp(params[6])
    p = np.exp(params[7])
    
    # Men's equations 
    hM = hazardM(λM, wstarM, α, μ, σ)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μ, σ, wstarM) ) )
    
    
    # Women's equations
    hF = hazardF(λF, wstarF, α, μ, σ, p, d)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    h = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μ, σ, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μ, σ, wstarF, d)

    i = np.sum( np.log( y + z ) )
    
    
    return -(a + b + c + e + f + g + h + i)

In [29]:
# Check on log-likelihood

param5 = [λ_M, λ_F, η_M, η_F, μ_0, σ_0, d_0, p_0]

b5_0 = np.log(param6[0])
b5_1 = np.log(param6[1])
b5_2 = np.log(param6[2])
b5_3 = np.log(param6[3])
# b5_4 = np.log(param6[4])
b5_4 = param6[4]
b5_5 = np.log(param6[5])
b5_6 = np.log(param6[6])
b5_7 = np.log(1)

init5 = [b5_0, b5_1, b5_2, b5_3, b5_4, b5_5, b5_6, b5_7]

print(loglik_5(init5))

14874.54168755806


In [30]:
est_5 = minimize(loglik_5, init5)

est_5

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Use

      fun: 6198.61640797023
 hess_inv: array([[ 3.63134359e-02, -1.45800634e-02,  3.14557389e-02,
        -2.12947544e-02, -5.26816135e-03,  2.96791910e-04,
        -9.35041045e-03,  2.53184764e-03],
       [-1.45800634e-02,  2.34732731e-02, -2.57176224e-02,
         1.78944043e-02, -7.98062634e-03,  3.01160140e-04,
        -1.11877513e-02,  2.56018645e-03],
       [ 3.14557389e-02, -2.57176224e-02,  7.33769966e-02,
        -3.37294017e-02, -6.73186116e-03,  4.11006791e-05,
        -9.35626935e-03,  3.61896418e-03],
       [-2.12947544e-02,  1.78944043e-02, -3.37294017e-02,
         4.61792714e-02, -8.50975056e-03,  2.40031002e-04,
        -1.47004772e-02,  4.93118894e-03],
       [-5.26816135e-03, -7.98062634e-03, -6.73186116e-03,
        -8.50975056e-03,  1.68250421e-02, -9.38015071e-04,
         4.26783993e-03, -3.38252164e-04],
       [ 2.96791910e-04,  3.01160140e-04,  4.11006791e-05,
         2.40031002e-04, -9.38015071e-04,  5.06859826e-04,
         1.16593961e-04,  6.92430763e-

In [31]:
# Coefficients

λM_5 = np.exp(est_5.x[0])
λF_5 = np.exp(est_5.x[1])
ηM_5 = np.exp(est_5.x[2])
ηF_5 = np.exp(est_5.x[3])
# μ_5 = np.exp(est_5.x[4])
μ_5 = est_5.x[4]
# σ_5 = np.exp(est_5.x[5])
σ_5 = est_5.x[5]
d_5 = np.exp(est_5.x[6])
p_5 = np.exp(est_5.x[7])

coeff_5 = [λM_5, λF_5, ηM_5, ηF_5, μ_5, σ_5, d_5, p_5]

print(coeff_5)

[0.20314888424069366, 0.2628387607190816, 0.0032968982534086373, 0.007672463396931774, 6.882380438061033, 3.0260896987852184, 3.7516201635059043, 0.42970113815027533]


In [32]:
lnL_5 = est_5.fun

print(lnL_5)

6198.61640797023


In [33]:
ts_5 = teststats(est_5.hess_inv, lnL_5, 8)

print(ts_5)

[[0.19056084571862936, 0.15320989896646833, 0.2708818867629654, 0.21489362814760402, 0.1297113799656998, 0.022513547609879114, 0.3341409806012453, 0.22044034700076987], [6198.61640797023, 5.947706417842323e-32]]


#### Estimation 2

In [34]:
def loglik_2( params: list ):
    """
    Calculates log likelihood with prejudice, no productivity differences (estimation 2).
    
    Parameters to estimate: λ, μ, σ, p, d
    """
    
    λM = np.exp(params[0])
    λF = np.exp(params[0])
    η = np.exp(params[1])
#     μ = np.exp(params[2])
    μ = params[2]
    σ = np.exp(params[3])
    d = np.exp(params[4])
    p = np.exp(params[5])

    
    # Men's equations 
    hM = hazardM(λM, wstarM, α, μ, σ)
    
    a = M['dur'].count() * np.log(hM/(hM+η))
    b = MU['dur'].count() * np.log(η)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μ, σ, wstarM) ) )
    
    
    # Women's equations
    hF = hazardF(λF, wstarF, α, μ, σ, p, d)
    
    f = F['dur'].count() * np.log(hF/(hF+η))
    g = FU['dur'].count() * np.log(η)
    i = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μ, σ, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μ, σ, wstarF, d)

    j = np.sum( np.log( y + z ) )
    
    
    return -(a + b + c + e + f + g + i + j)

In [35]:
# Check on log-likelihood

param2 = [λ_0, η_0, μ_0, σ_0, d_0, p_0]

b2_0 = np.log(param2[0])
b2_1 = np.log(param2[1])
# b2_2 = np.log(param2[2])
b2_2 = param2[2]
b2_3 = np.log(param2[3])
b2_4 = np.log(param2[4])
b2_5 = np.log(1)

init2 = [b2_0, b2_1, b2_2, b2_3, b2_4, b2_5]

print(loglik_2(init2))

16950.612216946025


In [36]:
est_2 = minimize(loglik_2, init2)

est_2

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log


      fun: 6200.846288388901
 hess_inv: array([[ 0.01359344,  0.00572998, -0.01359512,  0.00739644, -0.00707868,
         0.01125219],
       [ 0.00572998,  0.01496587, -0.00573208,  0.00308992, -0.00908033,
         0.00806914],
       [-0.01359512, -0.00573208,  0.01360975, -0.00740346,  0.00708786,
        -0.01126448],
       [ 0.00739644,  0.00308992, -0.00740346,  0.00649989, -0.01008112,
         0.01041043],
       [-0.00707868, -0.00908033,  0.00708786, -0.01008112,  0.16522995,
        -0.14861798],
       [ 0.01125219,  0.00806914, -0.01126448,  0.01041043, -0.14861798,
         0.14768418]])
      jac: array([6.10351562e-05, 0.00000000e+00, 6.10351562e-05, 1.22070312e-04,
       0.00000000e+00, 0.00000000e+00])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 647
      nit: 59
     njev: 80
   status: 2
  success: False
        x: array([-1.44716729, -5.25847417,  6.88239838,  3.02604807,  1.33519524,
       -0.85329763])

In [37]:
# Coefficients

λ_2 = np.exp(est_2.x[0])
η_2 = np.exp(est_2.x[1])
# μ_2 = np.exp(est_2.x[2])
# σ_2 = np.exp(est_2.x[3])
μ_2 = est_2.x[2]
σ_2 = est_2.x[3]
d_2 = np.exp(est_2.x[4])
p_2 = np.exp(est_2.x[5])

coeff_2 = [λ_2, η_2, μ_2, σ_2, d_2, p_2]

print(coeff_2)

[0.23523569902655178, 0.00520323792126242, 6.882398376933335, 3.026048066509593, 3.800737913025053, 0.4260077985422588]


In [38]:
lnL_2 = est_2.fun

print(lnL_2)

6200.846288388901


In [39]:
ts_2 = teststats(est_2.hess_inv, lnL_2, 6)

print(ts_2)

[[0.11659092468997184, 0.12233505515477239, 0.11666085150668226, 0.08062189722948172, 0.4064848702739455, 0.38429699408829693], [6200.846288388901, 2.448371368973035e-34]]


#### Estimation 4

In [63]:
def loglik_4( params: list ):
    """
    Calculates log likelihood with productivity differences, no prejudice (estimation 4).
    
    Parameters to estimate: λM, λF, μM, σM, μF, σF
    """

    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
    μM = params[4]
    σM = np.exp(params[5])
    μF = params[6]
    σF = np.exp(params[7])
    d = 0
    p = 0

    
    # Men's equations 
    hM = hazardM(λM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted( ME['wage'], α, μM, σM, wstarM) ) )
    
    # Women's equations
    hF = hazardF(λF, wstarF, α, μF, σF, p, d)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    i = - hF * np.sum(FU.values[:,0])
    
    y = (1/α) * dens_accepted( FE['wage'], α, μF, σF, wstarF)

    j = np.sum( np.log( y ) )

    return -(a + b + c + e + f + g + i + j)

In [64]:
# Check on log-likelihood

param4 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F]

b4_0 = np.log(param4[0])
b4_1 = np.log(param4[1])
b4_2 = np.log(param4[2])
b4_3 = np.log(param4[3])
b4_4 = param4[4]
b4_5 = np.log(param4[5])
b4_6 = param4[6]
b4_7 = np.log(param4[7])

init4 = [b4_0, b4_1, b4_2, b4_3, b4_4, b4_5, b4_6, b4_7]

print(loglik_4(init4))

15672.525637821565


In [65]:
print(init4)

[-1.7147984280919266, -1.2729656758128873, -5.809142990314028, -4.866534950122499, 3.456, -0.583396316600826, 3.454, -0.8603830999358592]


In [66]:
est_4 = minimize(loglik_4, init4)

est_4

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log


      fun: 6186.1919220010495
 hess_inv: array([[ 2.46493291e+01,  3.32569080e-03, -2.19745034e-02,
         1.07950290e-02,  8.76282533e-02, -9.72970590e-01,
        -7.52108068e-02,  5.08243229e-03],
       [ 3.32569080e-03,  2.81071491e-02,  3.69131585e-04,
         2.74942721e-02, -1.83974627e-04, -1.22514332e-04,
         2.93044438e-04, -1.67392525e-04],
       [-2.19745034e-02,  3.69131585e-04,  8.04006971e-02,
        -7.97549501e-04, -1.31850154e-03,  2.50478285e-03,
        -9.68454161e-04,  2.98653501e-04],
       [ 1.07950290e-02,  2.74942721e-02, -7.97549501e-04,
         5.47757977e-02, -1.33447006e-04, -4.42441006e-04,
         1.09867132e-04, -1.96615177e-04],
       [ 8.76282533e-02, -1.83974627e-04, -1.31850154e-03,
        -1.33447006e-04,  3.82958008e-04, -3.49252723e-03,
        -3.54265703e-04,  1.64115048e-05],
       [-9.72970590e-01, -1.22514332e-04,  2.50478285e-03,
        -4.42441006e-04, -3.49252723e-03,  3.84583240e-02,
         2.94363604e-03, -1.93791423

In [67]:
# Coefficients

λM_4 = np.exp(est_4.x[0])
λF_4 = np.exp(est_4.x[1])
ηM_4 = np.exp(est_4.x[2])
ηF_4 = np.exp(est_4.x[3])
μM_4 = est_4.x[4]
σM_4 = est_4.x[5]
μF_4 = est_4.x[6]
σF_4 = est_4.x[7]

coeff_4 = [λM_4, λF_4, ηM_4, ηF_4, μM_4, σM_4, μF_4, σF_4]

print(coeff_4)

[2.0360960512647076e+137, 0.260625178824421, 0.00328899410343176, 0.007674198534984502, -724.8905673016349, -18.44935070398948, 5.21724052582315, 2.8957801047095586]


In [68]:
lnL_4 = est_4.fun

print(lnL_4)

6186.1919220010495


In [69]:
ts_4 = teststats(est_4.hess_inv, lnL_4, 8)

print(ts_4)

[[4.9648090694403235, 0.1676518689019796, 0.28355016681938594, 0.23404229889119282, 0.01956931291405524, 0.1961079395637774, 0.12953035275294475, 0.03085324128629474], [6186.1919220010495, 9.078549630916258e-27]]


#### Estimation 1

In [57]:
def loglik_1( params: list ):
    """
    Calculates log likelihood with productivity differences, no prejudice (estimation 1).
    
    Parameters to estimate: λ, η, μM, σM, μF, σF
    """

    λM = np.exp(params[0]) #same lambda
    λF = np.exp(params[0]) #same lambda
    η = np.exp(params[1]) #same eta
#     μM = np.exp(params[2])
    μM = params[2]
    σM = np.exp(params[3])
#     σM = params[3]
#     μF = np.exp(params[4])
    μF = params[4]
    σF = np.exp(params[5])
#     σF = params[5]
    d = 0
    p = 0
    
    
    # Men's equations 
    hM = hazardM(λM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+η))
    b = MU['dur'].count() * np.log(η)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μM, σM, wstarM) ) )
    
    # Women's equations
    hF = hazardF(λM, wstarF, α, μF, σF, p, d)
    
    f = F['dur'].count() * np.log(hF/(hF+η))
    g = FU['dur'].count() * np.log(η)
    i = - hF * np.sum(FU.values[:,0])
    
    y = (1/α) * dens_accepted(FE['wage'], α, μF, σF, wstarF)
    
    j = np.sum( np.log( y ) )
    
    
    return -(a + b + c + e + f + g + i + j)

In [58]:
# Check on log-likelihood

param1 = [λ_0, η_0, μ_M, σ_M, μ_F, σ_F]

b1_0 = np.log(param1[0])
b1_1 = np.log(param1[1])
# b1_2 = np.log(param1[2])
b1_2 = param1[2]
b1_3 = np.log(param1[3])
# b1_3 = param1[3]
# b1_4 = np.log(param1[4])
b1_4 = param1[4]
b1_5 = np.log(param1[5])
# b1_5 = param1[5]

init1 = [b1_0, b1_1, b1_2, b1_3, b1_4, b1_5]

print(loglik_1(init1))

15762.081032459964


In [59]:
est_1 = minimize(loglik_1, init1)

est_1

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/

      fun: 6190.0344730251945
 hess_inv: array([[ 1.00984718e-01,  1.18408142e-01,  3.39585926e-02,
        -2.24676094e-04,  1.20694543e-02,  2.96089247e-03],
       [ 1.18408142e-01,  1.46984222e-01,  4.05265578e-02,
        -8.30437298e-04,  1.37073314e-02,  3.69506449e-03],
       [ 3.39585926e-02,  4.05265578e-02,  2.30683869e-02,
        -1.12074360e-03, -5.94482854e-03,  2.16202660e-03],
       [-2.24676094e-04, -8.30437298e-04, -1.12074360e-03,
         9.71210766e-04,  4.70240718e-04, -4.80293528e-05],
       [ 1.20694543e-02,  1.37073314e-02, -5.94482854e-03,
         4.70240718e-04,  2.44795733e-02, -1.35136018e-03],
       [ 2.96089247e-03,  3.69506449e-03,  2.16202660e-03,
        -4.80293528e-05, -1.35136018e-03,  1.34537335e-03]])
      jac: array([0., 0., 0., 0., 0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 508
      nit: 41
     njev: 62
   status: 0
  success: True
        x: array([-1.45175195, -5.25919617,  6.62778406,  3.15568414,  5.21753

In [60]:
# Coefficients

λ_1 = np.exp(est_1.x[0])
η_1 = np.exp(est_1.x[1])
# μM_1 = np.exp(est_1.x[2])
μM_1 = est_1.x[2]
# σM_1 = np.exp(est_1.x[3])
σM_1 = est_1.x[3]
# μF_1 = np.exp(est_1.x[4])
μF_1 = est_1.x[4]
# σF_1 = np.exp(est_1.x[5])
σF_1 = est_1.x[5]

coeff_1 = [λ_1, η_1, μM_1, σM_1, μF_1, σF_1]

print(coeff_1)

[0.2341596926727771, 0.0051994825566973836, 6.6277840630990035, 3.155684137396635, 5.217530541190654, 2.8958122600497362]


In [61]:
lnL_1 = est_1.fun

print(lnL_1)

6190.0344730251945


In [62]:
ts_1 = teststats(est_1.hess_inv, lnL_1, 6)

print(ts_1)

[[0.3177809279682816, 0.3833852138890972, 0.15188280661033646, 0.031164254612304, 0.15645949421809457, 0.03667933133752703], [6190.0344730251945, 9.248151564954754e-30]]


### Output to Latex

In [ ]:
np.arange(1, 10 )

In [ ]:
np.arange(10, 19)

In [ ]:
ts1 = [item for sublist in ts_1 for item in sublist]
ts2 = [item for sublist in ts_2 for item in sublist]
ts3 = [item for sublist in ts_3 for item in sublist]
ts4 = [item for sublist in ts_4 for item in sublist]
ts5 = [item for sublist in ts_5 for item in sublist]
ts6 = [item for sublist in ts_6 for item in sublist]

In [ ]:
# NA pad coef arrays
def pad(coeff: np.array):
    while len(coeff) != 11:
        coeff.append(None)
    return coeff

In [ ]:
# Pad all vectors for dataframe. 
pad(coeff_1)
pad(ts1)
pad(coeff_2)
pad(ts2)
pad(coeff_3)
pad(ts3)
pad(coeff_4)
pad(ts4)
pad(coeff_5)
pad(ts5)
pad(coeff_6)
pad(ts6)

In [ ]:
out = pd.DataFrame({
    '(1)': coeff_1,
    'se1': ts1,
    '(2)': coeff_2,
    'se2': ts2,
    '(3)': coeff_3,
    'se_3': ts3,
    '(4)': coeff_4,
    'se4': ts4,
    '(5)': coeff_5,
    'se5': ts5,
    '(6)': coeff_6,
    'se6': ts6
})

In [ ]:
np.savetxt('./output_megancode.csv', out, fmt='%1.4f', delimiter=',')

## Scratch

In [ ]:
def lambdaM(h: float, wstarM: float, α: float, μ: float, σ: float):
    """
    Estimates lambda for men
    """
    
    l = (α*μ) + ((1-α)*wstarM)
    s = α * σ
    shape = 1
    
    sf_in = (wstarM-l)/s
    
    denom = stats.lognorm.sf(sf_in, shape, l, s)
    
    return h/denom

In [ ]:
def lambdaF(h: float, wstarF: float, α: float, μ: float, σ: float, p: float, d: float):
    """
    Estimates lambda for women
    """
    
    l1 = (α*μ) + ((1-α)*wstarF)
    l2 = (α*μ) + ((1-α)*wstarF) - α*d
    s = α * σ
    shape = 1
    
    sf_in1 = (wstarF-l1)/s
    sf_in2 = (wstarF-l2)/s
    
    denom = (1-p)*stats.lognorm.sf(sf_in1, shape, l1, s) + p*stats.lognorm.sf(sf_in2, shape, l2, s)
    
    return h/denom

### Accepted Wage Function and Distributions as in 17, 18

In [ ]:
def wagefxn(wage: np.array, α: float, wstar: float):
    """
    Calculates wage (equation 8)
    """
    
    num = wage - (1-α)*wstar
    denom = α
    
    return num/denom

In [ ]:
def dens_accepted(wage: np.array, α: float, μ: float, σ: float, wstar: float):
    """
    Calculates the density of accepted wages using the lognormal distribution (eq 17 and first part of eq 18 in the paper)
    """
    
    l = (α*μ) + ((1-α)*wstar)
    s = α*σ
    shape = 1
    
    sf_in = (wstar-l)/s
    
    num = stats.lognorm.pdf(wage, shape, l, s) 
#     denom = stats.lognorm.sf(wstar, shape, l, s)
    denom = stats.lognorm.sf(sf_in, shape, loc=0, scale=1)
    
    return num / denom

In [ ]:
def dens_accepted_prej(wage: np.array, α: float, μ: float, σ: float, wstar: float, d: float):
    """
    Calculates the density of accepted wages when prejudice is present using the lognormal distribution (second part of eq 18 in the paper)
    """
    
    l = (α*μ) + ((1-α)*wstar) - (α*d)
    s = α*σ
    shape = 1
    
    sf_in = (wstar+d-l)/s
    
    num = stats.lognorm.pdf(wage,shape,l,s) 
#     denom = stats.lognorm.sf(wstar, shape, l, s) # +d or -d or no d?
    denom = stats.lognorm.sf(sf_in, shape, loc=0, scale=1)    
    
    return num / denom

In [ ]:
def hazardM(λ: float, wstar: float, α: float, μ: float, σ: float):
    """
    Estimates hazard rate for men (eq 10 of the paper)
    """
        
    l = (α*μ) + ((1-α)*wstar)
    s = α*σ
    shape = 1
    
    sf_in = (wstar-l)/s

#     mult = stats.lognorm.sf(wstar, shape, l, s)
    mult = stats.lognorm.sf(sf_in, shape, loc=0, scale=1)
    
    return λ * mult

In [ ]:
def hazardF(λ: float, wstar: float, α: float, μ: float, σ: float, p: float, d: float):
    """
    Estimates hazard rate for women (eq 10 of the paper)
    """
    
    l1 = (α*μ) + ((1-α)*wstar)
    l2 = (α*μ) + ((1-α)*wstar) - (α*d)
    s = α*σ
    shape = 1
    
    sf_in1 = (wstar-l1)/s
    sf_in2 = (wstar+d-l2)/s
    
#     mult = (1-p)*stats.lognorm.sf(wstar, shape, l1, s) + p*stats.lognorm.sf(wstar, shape, l2, s)
    mult = (1-p)*stats.lognorm.sf(sf_in1, shape, loc=0, scale=1) + p*stats.lognorm.sf(sf_in2, shape, loc=0, scale=1)
    
    return λ * mult